# Introduction to Clustering

**Overview:** In the [previous notebook](../how-to-get-baseball-data/extracting_baseball_data.ipynb) we learned how to extract baseball data from multiple sources using Python. Now we put that data to work. This notebook introduces **unsupervised learning** — specifically **clustering** — and applies the **K-Means** algorithm to real 2024 baseball data. We will cluster individual pitches to rediscover pitch types, group pitchers into archetypes based on their arsenals, and identify team playing styles across the league.

**Next Steps:** In the follow-up notebook (`advanced_clustering.ipynb`) we will explore additional clustering algorithms (DBSCAN, Hierarchical, Gaussian Mixture Models) and advanced evaluation techniques.

## Table of Contents

1. [Prerequisites & Setup](#1-prerequisites--setup)
2. [Supervised vs. Unsupervised Learning](#2-supervised-vs-unsupervised)
3. [Types of Unsupervised Learning](#3-types-of-unsupervised)
4. [What Is Clustering?](#4-what-is-clustering)
5. [The K-Means Algorithm](#5-k-means-algorithm)
6. [Toy Example: K-Means on Synthetic Data](#6-toy-example)
7. [Evaluating Clusters: Elbow Method & Silhouette Score](#7-evaluating-clusters)
8. [From Toy Data to Real Data: Feature Scaling](#8-toy-to-real)
9. [Baseball Example 1: Pitch Classification](#9-pitch-classification)
10. [Baseball Example 2: Pitcher Archetypes](#10-pitcher-archetypes)
11. [Baseball Example 3: Team Playing Styles](#11-team-playing-styles)
12. [Summary & Next Steps](#12-summary)

## 1. Prerequisites & Setup <a id="1-prerequisites--setup"></a>

We will build on the data extraction skills from the previous notebook and add machine learning and visualization libraries.

**Packages we'll use:**

| Package | Purpose |
|---------|----------|
| `pybaseball` | Statcast pitch data, FanGraphs stats |
| `pandas` | Data manipulation |
| `numpy` | Numerical operations |
| `scikit-learn` | K-Means clustering, StandardScaler, evaluation metrics |
| `matplotlib` | Static visualizations |
| `seaborn` | Statistical visualizations |
| `plotly` | Interactive visualizations (hover labels) |

**Requirements:**
- Python 3.8+
- An internet connection (we'll be pulling fresh data via pybaseball)

In [ ]:
# Install required packages
!pip install pybaseball pandas numpy scikit-learn matplotlib seaborn plotly

In [ ]:
# Base imports -- we'll import sklearn and plotly modules inline
# in each section so you can see exactly where they come from
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Enable pybaseball caching so re-runs are instant
from pybaseball import cache
cache.enable()

print("All base imports successful!")

## 2. Supervised vs. Unsupervised Learning <a id="2-supervised-vs-unsupervised"></a>

Machine learning is typically split into two broad categories based on whether or not you have **labeled data** — that is, data where the "right answer" is already known.

| | Supervised Learning | Unsupervised Learning |
|---|---|---|
| **Goal** | Predict a known label or outcome | Discover hidden structure in data |
| **Data** | Labeled — features + target variable | Unlabeled — features only |
| **Baseball example** | Predict if a pitch is a strike (label: ball/strike) | Group pitches by physical characteristics without knowing the pitch type |
| **Common algorithms** | Linear Regression, Random Forest, Neural Networks | K-Means, PCA, DBSCAN |

In supervised learning, you train a model on examples where the answer is provided, and then use that model to predict answers for new data. In unsupervised learning, there is **no answer key** — the algorithm's job is to find patterns and groupings on its own.

This notebook focuses entirely on unsupervised learning.

## 3. Types of Unsupervised Learning <a id="3-types-of-unsupervised"></a>

Unsupervised learning encompasses several different tasks:

- **Clustering**: Group similar items together. *Example: Group pitchers with similar arsenals into natural archetypes.*
- **Dimensionality Reduction**: Compress many features into fewer, more informative ones. *Example: Summarize 300+ Statcast columns into 2–3 components that capture most of the variation.* (Covered in Part 2 of this series.)
- **Anomaly Detection**: Identify unusual observations that don't fit the normal pattern. *Example: Flag a game where a pitcher's velocity drops 5 mph below his season average.*
- **Association Rules**: Discover which items or events tend to co-occur. *Example: Pitchers who throw sliders frequently tend to also have higher strikeout rates.*

This notebook focuses entirely on **clustering**. Dimensionality reduction is covered in Part 2.

## 4. What Is Clustering? <a id="4-what-is-clustering"></a>

**Clustering** is the task of partitioning data into groups (called **clusters**) such that items within a group are more similar to each other than to items in other groups.

**Intuition:** Imagine you have data on 1,000 pitches — just the speed and spin rate, with no labels telling you the pitch type. If you plotted speed vs. spin rate, you would probably see natural clumps: a group of fast, high-spin pitches (fastballs), a group of slower pitches with moderate spin (changeups), and a group with heavy break (curveballs). Clustering algorithms find these clumps automatically.

**When is clustering useful?**
- **Exploratory analysis**: What natural groupings exist in the data?
- **Player/team profiling**: Are there distinct "types" of hitters, pitchers, or teams?
- **Feature engineering**: Cluster assignments can become features for downstream supervised models
- **Segmentation**: Group similar items for tailored strategies (e.g., pitch sequencing against different batter types)

**Key property:** Clustering is unsupervised — you typically do **not** know the "right" number of groups or what those groups should be. The algorithm discovers the structure, and you bring domain knowledge to interpret it.

## 5. The K-Means Algorithm <a id="5-k-means-algorithm"></a>

**K-Means** is the most widely used clustering algorithm. It is simple, fast, and often the first tool to reach for when exploring your data.

### How K-Means Works (Step by Step)

1. **Choose K** — the number of clusters you want to find (you must decide this upfront)
2. **Initialize** — randomly place K points in the feature space. These are the initial **centroids** (cluster centers)
3. **Assign** — for each data point, compute the distance to every centroid and assign it to the **nearest** one
4. **Update** — move each centroid to the **mean** of all points currently assigned to it
5. **Repeat** steps 3–4 until the centroids stop moving (convergence)

The algorithm minimizes the **within-cluster sum of squares** (WCSS) — the total distance between each point and its assigned centroid.

### Key Properties

| Property | Detail |
|----------|--------|
| Requires K upfront | You must specify the number of clusters before running |
| Uses Euclidean distance | Features must be on similar scales (we'll cover scaling soon) |
| May find local optima | Different random initializations can give different results — scikit-learn's `n_init` parameter runs the algorithm multiple times and keeps the best |
| Fast | Time complexity is O(n × K × d × iterations), making it practical for large datasets |
| Assumes round clusters | Works best when clusters are roughly spherical and similar in size |

### Visualizing K-Means Step by Step

Let's watch the K-Means algorithm in action on a small dataset. We'll manually run through 4 iterations to see how centroids move and cluster assignments change.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# Generate a small dataset to visualize the algorithm
X_demo, _ = make_blobs(n_samples=50, centers=3, cluster_std=1.2, random_state=42)

fig, axes = plt.subplots(1, 4, figsize=(20, 4))

for i, max_iter in enumerate([1, 2, 3, 10]):
    # Run K-Means with a limited number of iterations
    km = KMeans(n_clusters=3, init='random', n_init=1, max_iter=max_iter, random_state=42)
    labels = km.fit_predict(X_demo)
    centroids = km.cluster_centers_

    ax = axes[i]
    ax.scatter(X_demo[:, 0], X_demo[:, 1], c=labels, cmap='viridis', s=30, alpha=0.7)
    ax.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200, edgecolors='black', linewidths=1.5)
    ax.set_title(f'Iteration {max_iter}' if max_iter < 10 else 'Converged')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

plt.suptitle('K-Means Algorithm: Step by Step', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()
print("Red X markers = centroids. Watch how they move toward the center of each cluster.")

## 6. Toy Example: K-Means on Synthetic Data <a id="6-toy-example"></a>

Before working with real baseball data, let's practice on synthetic data where we **know** the true clusters. This lets us verify that K-Means is working correctly.

In [ ]:
from sklearn.datasets import make_blobs

# Create 3 clusters of 2D data
X, y_true = make_blobs(n_samples=300, centers=3, cluster_std=0.8, random_state=42)

# Plot the raw data -- no labels, just points
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], s=30, alpha=0.7, color='gray')
plt.title('Synthetic Data — Can You See the Clusters?')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

print(f"Dataset: {X.shape[0]} points, {X.shape[1]} features")

In [ ]:
from sklearn.cluster import KMeans

# Apply K-Means with K=3
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(X)

labels = kmeans.labels_         # Cluster assignment for each point (0, 1, or 2)
centroids = kmeans.cluster_centers_  # Coordinates of the 3 centroids

print(f"Cluster assignments: {np.unique(labels)}")
print(f"Points per cluster: {np.bincount(labels)}")
print(f"Inertia (WCSS): {kmeans.inertia_:.2f}")

In [ ]:
# Side-by-side: K-Means clusters vs. ground truth
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: K-Means result
axes[0].scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=30, alpha=0.7)
axes[0].scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200,
                edgecolors='black', linewidths=1.5)
axes[0].set_title('K-Means Result (K=3)')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# Right: Ground truth
axes[1].scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=30, alpha=0.7)
axes[1].set_title('Ground Truth Labels')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')

plt.suptitle('K-Means recovered the true clusters!', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("Note: The cluster NUMBERS may differ (K-Means might call cluster 0 what the truth calls cluster 2),")
print("but the GROUPINGS should match.")

### What Happens with the Wrong K?

K-Means requires you to choose K upfront. Let's see what happens when we pick the wrong number of clusters.

In [ ]:
# Try different values of K
fig, axes = plt.subplots(1, 4, figsize=(20, 4))

for i, k in enumerate([2, 3, 4, 5]):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    pred = km.fit_predict(X)

    axes[i].scatter(X[:, 0], X[:, 1], c=pred, cmap='viridis', s=30, alpha=0.7)
    axes[i].scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
                    c='red', marker='X', s=200, edgecolors='black', linewidths=1.5)
    axes[i].set_title(f'K = {k}')
    axes[i].set_xlabel('Feature 1')
    axes[i].set_ylabel('Feature 2')

plt.suptitle('Effect of Choosing Different K Values', fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("K=2 merges two clusters. K=3 is just right. K=4 and K=5 split clusters unnecessarily.")
print("How do we pick the right K? That's next.")

## 7. Evaluating Clusters: Elbow Method & Silhouette Score <a id="7-evaluating-clusters"></a>

Since we usually don't have ground truth labels in unsupervised learning, we need other ways to assess how good our clusters are and how to choose K.

### 7a. The Elbow Method

The **Elbow Method** plots the **inertia** (within-cluster sum of squares) for different values of K. As K increases, inertia always decreases — but at some point the improvement becomes marginal. The "elbow" in the curve suggests a good K.

Think of it this way: going from 1 cluster to 2 dramatically reduces inertia. Going from 2 to 3 helps a lot. Going from 3 to 4? Not much improvement — you've already captured the natural structure.

In [ ]:
# Compute inertia for K = 1 through 10
inertias = []
K_range = range(1, 11)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    inertias.append(km.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.axvline(x=3, color='red', linestyle='--', alpha=0.7, label='Elbow at K=3')
plt.xlabel('Number of Clusters (K)', fontsize=12)
plt.ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
plt.title('Elbow Method', fontsize=13, fontweight='bold')
plt.legend()
plt.xticks(K_range)
plt.show()

print("The 'elbow' appears at K=3, confirming our synthetic data has 3 natural clusters.")

### 7b. Silhouette Score

The **Silhouette Score** measures how similar each point is to its own cluster compared to other clusters. It ranges from **-1 to +1**:

- **+1**: Points are well-matched to their cluster and poorly matched to neighbors
- **0**: Points are on the boundary between clusters
- **-1**: Points are likely assigned to the wrong cluster

A higher average silhouette score means better-defined clusters.

In [ ]:
from sklearn.metrics import silhouette_score

# Compute silhouette score for K = 2 through 8
sil_scores = []
K_sil_range = range(2, 9)  # Silhouette requires at least 2 clusters

for k in K_sil_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_k = km.fit_predict(X)
    sil_scores.append(silhouette_score(X, labels_k))

# Plot silhouette scores
plt.figure(figsize=(8, 5))
plt.plot(K_sil_range, sil_scores, 'go-', linewidth=2, markersize=8)
plt.axvline(x=3, color='red', linestyle='--', alpha=0.7, label='Best at K=3')
plt.xlabel('Number of Clusters (K)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Silhouette Score by K', fontsize=13, fontweight='bold')
plt.legend()
plt.xticks(K_sil_range)
plt.show()

print("Silhouette scores by K:")
for k, score in zip(K_sil_range, sil_scores):
    marker = " <-- best" if score == max(sil_scores) else ""
    print(f"  K={k}: {score:.4f}{marker}")

**Key takeaway:** Both the Elbow Method and Silhouette Score point to K=3 for our synthetic data — exactly right. In practice, these methods don't always agree perfectly, and domain knowledge plays an important role in the final decision. Use them as **guides**, not gospel.

## 8. From Toy Data to Real Data: Feature Scaling <a id="8-toy-to-real"></a>

Our toy example worked perfectly because both features were on similar scales. Real baseball data is different — features live on wildly different scales:

| Feature | Typical Range | Unit |
|---------|--------------|------|
| Pitch velocity | 70–100 | mph |
| Spin rate | 1,500–3,000 | rpm |
| Horizontal break | -2.0 to +2.0 | feet |
| Vertical break | -2.0 to +2.0 | feet |

K-Means uses **Euclidean distance** to assign points to clusters. Without scaling, a 100 rpm difference in spin rate would dominate a 5 mph difference in velocity — simply because the numbers are bigger. The algorithm would effectively ignore velocity.

**The fix: StandardScaler.** This transforms each feature to have mean = 0 and standard deviation = 1. After scaling, a 1-standard-deviation change in velocity is treated the same as a 1-standard-deviation change in spin rate.

**Rule of thumb:** Always scale your features before running K-Means.

### Other Considerations for Real Data

- **Higher dimensions**: Our toy example had 2 features we could plot directly. Real examples will have 4–10+ features. We'll use PCA to project clusters down to 2D for visualization. (PCA is covered in depth in Part 2.)
- **Missing data**: K-Means cannot handle NaN values. We'll need to drop or impute missing rows.
- **No ground truth**: In most real-world clustering, there are no labels to check against. Example 1 (Pitch Classification) is a lucky exception — Statcast labels every pitch.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Demo: StandardScaler in action
demo_df = pd.DataFrame({
    'velocity_mph': [95.2, 88.1, 92.4, 79.3, 96.7],
    'spin_rate_rpm': [2400, 2650, 2200, 2800, 2350]
})

scaler = StandardScaler()
scaled = scaler.fit_transform(demo_df)
scaled_df = pd.DataFrame(scaled, columns=demo_df.columns)

print("BEFORE scaling:")
print(demo_df.to_string(index=False))
print(f"\n  velocity range: {demo_df['velocity_mph'].min():.1f} to {demo_df['velocity_mph'].max():.1f}")
print(f"  spin rate range: {demo_df['spin_rate_rpm'].min():.0f} to {demo_df['spin_rate_rpm'].max():.0f}")

print("\nAFTER scaling (mean=0, std=1):")
print(scaled_df.round(2).to_string(index=False))
print(f"\n  velocity range: {scaled_df['velocity_mph'].min():.2f} to {scaled_df['velocity_mph'].max():.2f}")
print(f"  spin rate range: {scaled_df['spin_rate_rpm'].min():.2f} to {scaled_df['spin_rate_rpm'].max():.2f}")
print("\nNow both features are on the same scale!")

## 9. Baseball Example 1: Pitch Classification <a id="9-pitch-classification"></a>

Can K-Means rediscover pitch types from raw tracking data? Statcast labels every pitch with a `pitch_type` (fastball, slider, curveball, etc.), but what if we stripped those labels away and asked the algorithm to find groups based only on physical characteristics?

We'll use **Tarik Skubal's** 2024 season — the AL Cy Young winner we featured in the data extraction notebook. He throws 4 distinct pitch types which should give us a clear result.

In [ ]:
from pybaseball import statcast_pitcher, playerid_lookup

# Look up Tarik Skubal's player ID
skubal_info = playerid_lookup('skubal', 'tarik')
skubal_id = skubal_info['key_mlbam'].values[0]
print(f"Skubal's MLBAM ID: {skubal_id}")

# Pull his full 2024 season of Statcast data
skubal_data = statcast_pitcher(
    start_dt='2024-03-28',
    end_dt='2024-09-29',
    player_id=skubal_id
)

print(f"\nShape: {skubal_data.shape[0]:,} pitches x {skubal_data.shape[1]} columns")
skubal_data.head()

In [ ]:
# Select our clustering features + the pitch_type label for later comparison
feature_cols = ['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']

pitch_df = skubal_data[feature_cols + ['pitch_type']].copy()

# Check missing values
print("Missing values:")
print(pitch_df.isnull().sum())

print(f"\nPitch type breakdown:")
print(pitch_df['pitch_type'].value_counts())

In [ ]:
# Preprocessing: drop missing values and scale features
pitch_clean = pitch_df.dropna(subset=feature_cols).copy()
print(f"After dropping NaN: {len(pitch_clean):,} pitches ({len(pitch_df) - len(pitch_clean)} removed)")

# Filter out very rare pitch types (fewer than 10 occurrences)
type_counts = pitch_clean['pitch_type'].value_counts()
common_types = type_counts[type_counts >= 10].index
pitch_clean = pitch_clean[pitch_clean['pitch_type'].isin(common_types)]
print(f"After filtering rare pitch types: {len(pitch_clean):,} pitches")
print(f"Pitch types kept: {list(common_types)}")

# Scale the features
scaler_pitch = StandardScaler()
X_pitch = scaler_pitch.fit_transform(pitch_clean[feature_cols])
print(f"\nScaled feature matrix shape: {X_pitch.shape}")

In [ ]:
# Determine K using Elbow Method and Silhouette Score
inertias_pitch = []
sil_scores_pitch = []
K_range_pitch = range(2, 9)

for k in K_range_pitch:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    pred = km.fit_predict(X_pitch)
    inertias_pitch.append(km.inertia_)
    sil_scores_pitch.append(silhouette_score(X_pitch, pred))

# Plot both
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(K_range_pitch, inertias_pitch, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (K)')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Elbow Method — Skubal Pitches')
axes[0].set_xticks(list(K_range_pitch))

axes[1].plot(K_range_pitch, sil_scores_pitch, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (K)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score — Skubal Pitches')
axes[1].set_xticks(list(K_range_pitch))

plt.tight_layout()
plt.show()

print("Silhouette scores:")
for k, score in zip(K_range_pitch, sil_scores_pitch):
    print(f"  K={k}: {score:.4f}")

In [ ]:
# Fit K-Means with the number of pitch types we found
n_pitch_types = len(common_types)
print(f"Using K={n_pitch_types} (matching the number of actual pitch types)")

kmeans_pitch = KMeans(n_clusters=n_pitch_types, random_state=42, n_init=10)
pitch_clean['cluster'] = kmeans_pitch.fit_predict(X_pitch)

print(f"\nPoints per cluster:")
print(pitch_clean['cluster'].value_counts().sort_index())

In [ ]:
# Visualization 1: Velocity vs Spin Rate colored by cluster
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# K-Means clusters
sns.scatterplot(data=pitch_clean, x='release_speed', y='release_spin_rate',
                hue='cluster', palette='viridis', alpha=0.6, s=20, ax=axes[0], legend='full')
axes[0].set_title('K-Means Clusters')
axes[0].set_xlabel('Velocity (mph)')
axes[0].set_ylabel('Spin Rate (rpm)')

# Actual pitch types
sns.scatterplot(data=pitch_clean, x='release_speed', y='release_spin_rate',
                hue='pitch_type', palette='Set2', alpha=0.6, s=20, ax=axes[1], legend='full')
axes[1].set_title('Actual Pitch Types')
axes[1].set_xlabel('Velocity (mph)')
axes[1].set_ylabel('Spin Rate (rpm)')

plt.suptitle('Velocity vs. Spin Rate — Clusters vs. Reality', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Visualization 2: Pitch Movement Plot (pfx_x vs pfx_z) — the baseball classic
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# K-Means clusters
sns.scatterplot(data=pitch_clean, x='pfx_x', y='pfx_z',
                hue='cluster', palette='viridis', alpha=0.6, s=20, ax=axes[0], legend='full')
axes[0].set_title('K-Means Clusters')
axes[0].set_xlabel('Horizontal Break (ft)')
axes[0].set_ylabel('Vertical Break (ft)')
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.3)
axes[0].axvline(x=0, color='gray', linestyle='--', alpha=0.3)

# Actual pitch types
sns.scatterplot(data=pitch_clean, x='pfx_x', y='pfx_z',
                hue='pitch_type', palette='Set2', alpha=0.6, s=20, ax=axes[1], legend='full')
axes[1].set_title('Actual Pitch Types')
axes[1].set_xlabel('Horizontal Break (ft)')
axes[1].set_ylabel('Vertical Break (ft)')
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.3)
axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.3)

plt.suptitle('Pitch Movement — Clusters vs. Reality', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print("This is the classic 'pitch movement' plot. Each clump represents a different pitch type.")
print("Compare left vs. right — K-Means found the same groups!")

In [ ]:
# Interactive plotly scatter — hover over any pitch to see details
import plotly.express as px

fig = px.scatter(
    pitch_clean,
    x='pfx_x', y='pfx_z',
    color='cluster',
    hover_data=['pitch_type', 'release_speed', 'release_spin_rate'],
    title='Interactive: Pitch Movement Colored by K-Means Cluster',
    labels={
        'pfx_x': 'Horizontal Break (ft)',
        'pfx_z': 'Vertical Break (ft)',
        'release_speed': 'Velocity (mph)',
        'release_spin_rate': 'Spin Rate (rpm)',
        'cluster': 'Cluster'
    },
    opacity=0.6,
    color_continuous_scale='Viridis'
)
fig.update_layout(width=700, height=550)
fig.show()

print("Hover over any point to see its actual pitch type, velocity, and spin rate!")

In [ ]:
# How well did K-Means match reality? Cross-tabulation
ct = pd.crosstab(pitch_clean['pitch_type'], pitch_clean['cluster'], margins=True)
print("Cross-tabulation: Actual Pitch Type vs. K-Means Cluster\n")
print(ct)

# Compute "accuracy" by mapping each cluster to its majority pitch type
print("\n--- Cluster-to-Pitch-Type Mapping ---")
correct = 0
for cluster_id in range(n_pitch_types):
    cluster_mask = pitch_clean['cluster'] == cluster_id
    majority_type = pitch_clean.loc[cluster_mask, 'pitch_type'].mode()[0]
    count_correct = (pitch_clean.loc[cluster_mask, 'pitch_type'] == majority_type).sum()
    count_total = cluster_mask.sum()
    correct += count_correct
    print(f"  Cluster {cluster_id} -> {majority_type} ({count_correct}/{count_total} = {count_correct/count_total:.1%})")

overall_acc = correct / len(pitch_clean)
print(f"\nOverall accuracy: {correct}/{len(pitch_clean)} = {overall_acc:.1%}")

**Key takeaway:** K-Means was able to rediscover Skubal's pitch types purely from physical measurements — velocity, spin rate, and movement — with no labels provided. This is the power of clustering: finding natural structure in data. Where the algorithm struggles (e.g., separating fastballs from sinkers) reflects genuine physical similarity between those pitch types.

## 10. Baseball Example 2: Pitcher Archetypes <a id="10-pitcher-archetypes"></a>

Instead of clustering individual pitches, let's zoom out and cluster **pitchers**. Every pitcher has a unique profile: some rely on a dominant fastball with elite velocity, others use a diverse arsenal of breaking balls. Can K-Means discover natural pitcher archetypes from season-level stats?

In [ ]:
from pybaseball import pitching_stats

# Get 2024 pitching stats from FanGraphs
# qual=50 gives us pitchers with at least 50 IP for a broader sample
pitching_2024 = pitching_stats(2024, qual=50)

print(f"Shape: {pitching_2024.shape[0]} pitchers x {pitching_2024.shape[1]} columns")
pitching_2024.head()

In [ ]:
# Let's explore available columns related to our features of interest
# We want: velocity, strikeout/walk rates, batted ball profile, pitch mix
all_cols = pitching_2024.columns.tolist()

# Print columns that might contain our desired features
print("Velocity-related columns:")
print([c for c in all_cols if 'vFA' in c or 'FBv' in c or 'Velocity' in c or c.startswith('v')][:15])

print("\nRate columns (K%, BB%, etc.):")
print([c for c in all_cols if '%' in c][:20])

print("\nPitch mix / usage columns:")
print([c for c in all_cols if c.endswith('%') and len(c) <= 5][:15])

In [ ]:
# Select features for clustering
# We'll pick features that capture velocity, command, contact quality, and pitch mix
# NOTE: Column names may vary slightly across pybaseball versions.
# If a column is missing, check the printed columns above and adjust.

pitcher_feature_cols = [
    'K%',       # Strikeout rate -- swing-and-miss ability
    'BB%',      # Walk rate -- command/control
    'GB%',      # Ground ball rate -- contact management
    'HR/FB',    # Home runs per fly ball -- fly ball damage
    'SwStr%',   # Swinging strike rate -- stuff quality
]

# Check which columns actually exist and add velocity if available
for vel_col in ['vFA (pi)', 'FBv', 'vFA']:
    if vel_col in pitching_2024.columns:
        pitcher_feature_cols.insert(0, vel_col)
        print(f"Using '{vel_col}' for fastball velocity")
        break

# Verify all selected columns exist
missing = [c for c in pitcher_feature_cols if c not in pitching_2024.columns]
if missing:
    print(f"WARNING: Missing columns: {missing}")
    pitcher_feature_cols = [c for c in pitcher_feature_cols if c in pitching_2024.columns]

print(f"\nFinal feature set ({len(pitcher_feature_cols)} features): {pitcher_feature_cols}")

In [ ]:
# Filter to starting pitchers for a more homogeneous comparison
starters = pitching_2024[pitching_2024['GS'] >= 10].copy()
print(f"Starting pitchers (GS >= 10): {len(starters)}")

# Preprocessing: drop missing values and scale
pitcher_clean = starters[['Name', 'Team', 'WAR', 'ERA', 'IP'] + pitcher_feature_cols].dropna(subset=pitcher_feature_cols).copy()
print(f"After dropping NaN: {len(pitcher_clean)} pitchers")

scaler_pitcher = StandardScaler()
X_pitcher = scaler_pitcher.fit_transform(pitcher_clean[pitcher_feature_cols])
print(f"Scaled feature matrix: {X_pitcher.shape}")

# Quick look at the data
pitcher_clean[['Name', 'Team', 'WAR'] + pitcher_feature_cols].head(10)

In [ ]:
# Determine K with Elbow and Silhouette
inertias_p = []
sil_scores_p = []
K_range_p = range(2, 9)

for k in K_range_p:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    pred = km.fit_predict(X_pitcher)
    inertias_p.append(km.inertia_)
    sil_scores_p.append(silhouette_score(X_pitcher, pred))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(K_range_p, inertias_p, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (K)')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Elbow Method — Pitcher Archetypes')
axes[0].set_xticks(list(K_range_p))

axes[1].plot(K_range_p, sil_scores_p, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (K)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score — Pitcher Archetypes')
axes[1].set_xticks(list(K_range_p))

plt.tight_layout()
plt.show()

print("Silhouette scores:")
for k, score in zip(K_range_p, sil_scores_p):
    print(f"  K={k}: {score:.4f}")

In [ ]:
# Fit K-Means — choose K based on the elbow/silhouette plots above
# We'll use K=4 as a reasonable default; adjust if your plots suggest otherwise
K_pitcher = 4
kmeans_pitcher = KMeans(n_clusters=K_pitcher, random_state=42, n_init=10)
pitcher_clean['archetype'] = kmeans_pitcher.fit_predict(X_pitcher)

print(f"Using K={K_pitcher}")
print(f"\nPitchers per archetype:")
print(pitcher_clean['archetype'].value_counts().sort_index())

In [ ]:
# Visualization 1: Cluster profile heatmap
# Shows the mean (scaled) feature value for each archetype
cluster_means = pitcher_clean.groupby('archetype')[pitcher_feature_cols].mean()

# Scale the means for better heatmap visualization
cluster_means_scaled = pd.DataFrame(
    StandardScaler().fit_transform(cluster_means),
    index=cluster_means.index,
    columns=cluster_means.columns
)

plt.figure(figsize=(10, 5))
sns.heatmap(cluster_means_scaled, annot=cluster_means.round(1).values, fmt='',
            cmap='RdYlBu_r', center=0, linewidths=0.5,
            yticklabels=[f'Archetype {i}' for i in range(K_pitcher)])
plt.title('Pitcher Archetype Profiles (values = raw means, colors = relative scale)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("Each row is an archetype. Warm colors = above average, cool colors = below average.")

In [ ]:
# Visualization 2: PCA projection to 2D for an interactive scatter
from sklearn.decomposition import PCA

# Project the pitcher features down to 2 dimensions
# (We'll cover PCA in detail in Part 2 -- for now, think of it as
#  compressing our features into 2 axes that capture the most variation)
pca_pitcher = PCA(n_components=2)
X_pitcher_2d = pca_pitcher.fit_transform(X_pitcher)

pitcher_clean['PC1'] = X_pitcher_2d[:, 0]
pitcher_clean['PC2'] = X_pitcher_2d[:, 1]

fig = px.scatter(
    pitcher_clean,
    x='PC1', y='PC2',
    color='archetype',
    hover_data=['Name', 'Team', 'WAR', 'ERA'],
    title='Pitcher Archetypes — PCA Projection (hover for names)',
    labels={'PC1': f'PC1 ({pca_pitcher.explained_variance_ratio_[0]:.1%} variance)',
            'PC2': f'PC2 ({pca_pitcher.explained_variance_ratio_[1]:.1%} variance)'},
    opacity=0.7,
)
fig.update_layout(width=750, height=550)
fig.show()

print(f"PCA explains {pca_pitcher.explained_variance_ratio_.sum():.1%} of total variance in 2 components.")
print("Hover over any point to see the pitcher's name, team, WAR, and ERA!")

In [ ]:
# Interpret archetypes: top pitchers in each cluster
print("=" * 70)
print("PITCHER ARCHETYPES — Top 5 by WAR in each cluster")
print("=" * 70)

for archetype_id in range(K_pitcher):
    members = pitcher_clean[pitcher_clean['archetype'] == archetype_id]

    # Summarize the archetype's profile
    profile = members[pitcher_feature_cols].mean()

    print(f"\n--- Archetype {archetype_id} ({len(members)} pitchers) ---")
    print(f"Profile: {', '.join(f'{col}={val:.1f}' for col, val in profile.items())}")
    print()
    print(members.nlargest(5, 'WAR')[['Name', 'Team', 'WAR', 'ERA'] + pitcher_feature_cols].to_string(index=False))

print("\n" + "=" * 70)
print("Look at each archetype's profile and notable pitchers.")
print("Can you give each archetype a descriptive name? (e.g., 'Power Arms', 'Craftsmen', 'Groundballers')")

**Key takeaway:** K-Means can discover natural pitcher archetypes from season-level statistics. Unlike the pitch classification example, there is no "ground truth" here — the value comes from the archetypes themselves and the insights they provide about different pitching profiles. The heatmap and archetype members help us interpret and name the clusters.

## 11. Baseball Example 3: Team Playing Styles <a id="11-team-playing-styles"></a>

Finally, let's zoom all the way out and cluster **entire teams**. Do teams have distinct strategic identities? With only 30 MLB teams, every cluster will contain recognizable names — making the results easy to discuss and debate.

In [ ]:
from pybaseball import team_batting, team_pitching

# Pull 2024 team-level stats
team_bat = team_batting(2024)
team_pitch = team_pitching(2024)

print(f"Team batting: {team_bat.shape}")
print(f"Team pitching: {team_pitch.shape}")

team_bat.head()

In [ ]:
# Merge batting and pitching on team
# Rename colliding columns to avoid confusion
team_bat_renamed = team_bat.rename(columns={'K%': 'K%_bat', 'BB%': 'BB%_bat'})
team_pitch_renamed = team_pitch.rename(columns={'K%': 'K%_pitch', 'BB%': 'BB%_pitch'})

# Find the common team identifier column
# (pybaseball may use 'Team' or 'teamID' -- let's check)
print("Batting columns:", team_bat_renamed.columns[:10].tolist())
print("Pitching columns:", team_pitch_renamed.columns[:10].tolist())

# Merge on the team name column
team_col = 'Team' if 'Team' in team_bat_renamed.columns else team_bat_renamed.columns[0]
teams = team_bat_renamed.merge(team_pitch_renamed, on=team_col, suffixes=('_bat', '_pitch'))
print(f"\nMerged teams: {len(teams)} rows")
teams.head()

In [ ]:
# Select features for team clustering
# We want a mix of offensive and pitching characteristics
team_feature_candidates = {
    # Offense
    'HR': 'HR',
    'SB': 'SB',
    'K%_bat': 'K%_bat',
    'BB%_bat': 'BB%_bat',
    'OPS': 'OPS',
    # Pitching
    'ERA': 'ERA',
    'K%_pitch': 'K%_pitch',
    'BB%_pitch': 'BB%_pitch',
}

# Use whichever columns exist in the merged dataframe
team_feature_cols = [c for c in team_feature_candidates.values() if c in teams.columns]
print(f"Team features ({len(team_feature_cols)}): {team_feature_cols}")

# Check for missing values
team_clean = teams[[team_col] + team_feature_cols].dropna(subset=team_feature_cols).copy()
print(f"Teams with complete data: {len(team_clean)}")

# Scale
scaler_team = StandardScaler()
X_team = scaler_team.fit_transform(team_clean[team_feature_cols])
print(f"Scaled feature matrix: {X_team.shape}")

In [ ]:
# With only 30 teams, we'll skip the elbow/silhouette (already demonstrated)
# and jump straight to K=4 for interpretability
K_team = 4
kmeans_team = KMeans(n_clusters=K_team, random_state=42, n_init=10)
team_clean['cluster'] = kmeans_team.fit_predict(X_team)

# PCA for 2D visualization
pca_team = PCA(n_components=2)
X_team_2d = pca_team.fit_transform(X_team)
team_clean['PC1'] = X_team_2d[:, 0]
team_clean['PC2'] = X_team_2d[:, 1]

# Interactive scatter — find your favorite team!
fig = px.scatter(
    team_clean,
    x='PC1', y='PC2',
    color='cluster',
    text=team_col,
    hover_data=team_feature_cols,
    title='MLB Team Playing Styles (2024) — Find Your Team!',
    labels={'PC1': f'PC1 ({pca_team.explained_variance_ratio_[0]:.1%} variance)',
            'PC2': f'PC2 ({pca_team.explained_variance_ratio_[1]:.1%} variance)'},
    opacity=0.8,
)
fig.update_traces(textposition='top center', textfont_size=9)
fig.update_layout(width=800, height=600)
fig.show()

print("Hover over any team to see their stats. Teams in the same cluster have similar profiles.")

In [ ]:
# Cluster profiles and team membership
print("=" * 60)
print("TEAM PLAYING STYLE CLUSTERS")
print("=" * 60)

for cluster_id in range(K_team):
    members = team_clean[team_clean['cluster'] == cluster_id]
    profile = members[team_feature_cols].mean()

    print(f"\n--- Cluster {cluster_id} ({len(members)} teams) ---")
    print(f"Teams: {', '.join(members[team_col].values)}")
    print(f"Avg profile:")
    for col in team_feature_cols:
        val = profile[col]
        print(f"  {col:15s} {val:.2f}")

print("\n" + "=" * 60)
print("Can you name each cluster? (e.g., 'Power Offenses', 'Pitching-First', 'Rebuilding')")

**Key takeaway:** Even with just 30 data points, K-Means can identify meaningful team playing styles. The same clustering workflow — select features, scale, fit, visualize, interpret — works at every level of granularity: individual pitches, player careers, and team identities.

## 12. Summary & Next Steps <a id="12-summary"></a>

### What We Covered

| Example | Data Source | Features | K | Key Finding |
|---------|------------|----------|---|-------------|
| Pitch Classification | `statcast_pitcher()` | velocity, spin, movement | ~4 | K-Means recovered actual pitch types from raw data |
| Pitcher Archetypes | `pitching_stats()` | velocity, rates, pitch mix | ~4 | Natural groupings like Power Arms and Craftsmen |
| Team Playing Styles | `team_batting()` + `team_pitching()` | offense + pitching stats | 4 | Teams cluster by strategic identity |

### Key Lessons

1. **K-Means is a powerful first-pass tool** for discovering structure in data
2. **Feature scaling is critical** — always use `StandardScaler` before K-Means
3. **The Elbow Method and Silhouette Score** help choose K, but domain knowledge is equally important
4. **Clustering is unsupervised** — there's rarely a "right answer," and interpretation requires context
5. **The same workflow applies at every level** — from individual pitches to entire teams

### What's Next

**Advanced Clustering** (`advanced_clustering.ipynb`):
- **DBSCAN**: A clustering algorithm that doesn't require specifying K and can find oddly shaped clusters and outliers
- **Hierarchical Clustering**: Builds a tree (dendrogram) of nested clusters, giving you flexibility in choosing the number of groups
- **Gaussian Mixture Models**: A probabilistic approach that assigns soft/fuzzy cluster memberships instead of hard assignments
- **Advanced evaluation**: Davies-Bouldin index, Calinski-Harabasz score, Gap Statistic

**Unsupervised Learning Part 2 — Dimensionality Reduction**:
- **PCA**: Compress many features into fewer components (we used it briefly for visualization here)
- **t-SNE and UMAP**: Non-linear reduction techniques for beautiful 2D visualizations
- **Combining reduction + clustering**: Reduce first, then cluster for better results

Happy clustering!